# Pandas

## Лабораторная работа №2
Выполнил: Студент группы ДПИ22-1 Сизов Дмитрий

### Базовые операции с `DataFrame`

1.1 В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца с индексами в таблице `reviews` (безымянный столбец).

In [1]:
import pandas as pd
recipes = pd.read_csv('data/recipes_sample.csv')
reviews = pd.read_csv('data/reviews_sample.csv')

1.2 Для каждой из таблиц выведите основные параметры:
* количество точек данных (строк);
* количество столбцов;
* тип данных каждого столбца.

In [2]:
tables = [('Recipes', recipes), ('Reviews', reviews)]
for name, table in tables:
    print(name)
    print(f'Строк: {len(table.index)}')
    print(f'Столбцов: {len(table.columns)}')
    print(table.dtypes)


Recipes
Строк: 30000
Столбцов: 8
name               object
id                  int64
minutes             int64
contributor_id      int64
submitted          object
n_steps           float64
description        object
n_ingredients     float64
dtype: object
Reviews
Строк: 126696
Столбцов: 6
Unnamed: 0     int64
user_id        int64
recipe_id      int64
date          object
rating         int64
review        object
dtype: object


1.3 Исследуйте, в каких столбцах таблиц содержатся пропуски. Посчитайте долю строк, содержащих пропуски, в отношении к общему количеству строк.

In [3]:
import numpy as np

for _, table in tables:
    print(round(np.count_nonzero(table.isnull()) / len(table.index) * 100, 2))

68.98
0.01


1.4 Рассчитайте среднее значение для каждого из числовых столбцов (где это имеет смысл).

In [4]:
print(recipes.loc[:, 'minutes'].mean())
print(recipes.loc[:, 'n_steps'].mean())
print(recipes.loc[:, 'n_ingredients'].mean())
print(reviews.loc[:, 'rating'].mean())

123.35813333333333
9.805582137161085
9.008285984848484
4.410802235271832


1.5 Создайте серию из 10 случайных названий рецептов.

In [5]:
print(list(recipes.sample(10).loc[:, 'name']))

['tasty chicken drumettes  kid pleaser', 'easy au gratin s', 'banana split bites', 'no cook divinity', 'cinnamon applesauce jello', 'strawberry blender mousse', 'citrus roasted fish with capers', 'cheesy stuffing cups', 'mama mia s caesar salad', 'caramel tassie cookies']


1.6 Измените индекс в таблице `reviews`, пронумеровав строки, начиная с нуля.

1.7 Выведите информацию о рецептах, время выполнения которых не больше 20 минут и кол-во ингредиентов в которых не больше 5.

In [6]:
print(recipes[(recipes.minutes < 20) & (recipes.n_ingredients < 5)])

                                             name      id  minutes  \
90              hawaiian sunrise           mimosa  100837        5   
91     tasty dish s   banana pudding in 2 minutes  286484        2   
94                             1 minute meatballs   11361       13   
143                             2 minute broccoli  256464        4   
168                 3 ingredient moroccan dry rub  505748        3   
...                                           ...     ...      ...   
29817                           zen wine spritzer  498523        1   
29858                        zesty stuffed olives  269904       10   
29866                   zhanyshpa from kazakhstan  160599       15   
29874                   ziplock vanilla ice cream   74250       10   
29980        zucchini with jalapeno monterey jack  320622       10   

       contributor_id   submitted  n_steps  \
90              58104  2004-09-29      4.0   
91              47892  2008-02-13      NaN   
94               4470

### Работа с датами в `pandas`

2.1 Преобразуйте столбец `submitted` из таблицы `recipes` в формат времени. Модифицируйте решение задачи 1.1 так, чтобы считать столбец сразу в нужном формате.

In [7]:
recipes.submitted = pd.to_datetime(recipes.submitted)

2.2 Выведите информацию о рецептах, добавленных в датасет не позже 2010 года.

In [8]:
print(recipes[recipes.submitted.dt.year < 2010])

                                               name      id  minutes  \
0             george s at the cove  black bean soup   44123       90   
1                healthy for them  yogurt popsicles   67664       10   
2                      i can t believe it s spinach   38798       30   
3                              italian  gut busters   35173       45   
4          love is in the air  beef fondue   sauces   84797       25   
...                                             ...     ...      ...   
29993                     zuni caf zucchini pickles  316950     2895   
29995  zurie s holey rustic olive and cheddar bread  267661       80   
29996          zwetschgenkuchen  bavarian plum cake  386977      240   
29997   zwiebelkuchen   southwest german onion cake  103312       75   
29999        cookies by design   cookies on a stick  298512       29   

       contributor_id  submitted  n_steps  \
0               35193 2002-10-25      NaN   
1               91970 2003-07-26      NaN   


### Работа со строковыми данными в `pandas`

3.1  Добавьте в таблицу `recipes` столбец `description_length`, в котором хранится длина описания рецепта из столбца `description`.

In [9]:
recipes['description_length'] = recipes['description'].str.len()

3.2 Измените название каждого рецепта в таблице `recipes` таким образом, чтобы каждое слово в названии начиналось с прописной буквы.

In [10]:
recipes['name'] = recipes['name'].str.capitalize()

3.3 Добавьте в таблицу `recipes` столбец `name_word_count`, в котором хранится количество слов из названии рецепта (считайте, что слова в названии разделяются только пробелами). Обратите внимание, что между словами может располагаться несколько пробелов подряд.

In [11]:
recipes['name_word_count'] = recipes['name'].str.split().str.len()

### Группировки таблиц `pd.DataFrame`

4.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [12]:
grouped_recipes = recipes.groupby('contributor_id')
print(grouped_recipes.size())
print(grouped_recipes.size().idxmax())

contributor_id
1530            5
1533          186
1534           50
1535           40
1538            8
             ... 
2001968497      2
2002059754      1
2002234079      1
2002234259      1
2002247884      1
Length: 8404, dtype: int64
89831


4.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [13]:
grouped_reviews = reviews.groupby('recipe_id')
print(grouped_reviews['rating'].mean())
print(len(recipes.index) - len(grouped_reviews))

recipe_id
48        1.000000
55        4.750000
66        4.944444
91        4.750000
94        5.000000
            ...   
536547    5.000000
536610    0.000000
536728    4.000000
536729    4.750000
536747    0.000000
Name: rating, Length: 28100, dtype: float64
1900


4.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [14]:
print(recipes.groupby(recipes.submitted.dt.year).size())

submitted
1999     275
2000     104
2001     589
2002    2644
2003    2334
2004    2153
2005    3130
2006    3473
2007    4429
2008    4029
2009    2963
2010    1538
2011     922
2012     659
2013     490
2014     139
2015      42
2016      24
2017      39
2018      24
dtype: int64


### Объединение таблиц `pd.DataFrame`

5.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [15]:
recipes_merged = recipes[['id', 'name']].copy()
reviews_temp = reviews[['recipe_id', 'user_id', 'rating']].copy()
recipes_merged = pd.merge(recipes_merged, reviews_temp, how='inner', left_on='id', right_on='recipe_id').drop(columns=['recipe_id'])
for recipe in recipes.id:
    if recipe not in reviews.recipe_id:
        print(recipe in recipes_merged.id)
        break

False


5.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [16]:
recipes_temp = recipes[['id', 'name']].copy()
reviews_merged = reviews[['recipe_id']].copy()
reviews_merged = pd.merge(recipes_temp, reviews_merged, how='right', left_on='id', right_on='recipe_id').drop(columns=['id'])
review_count = reviews_merged.groupby('recipe_id', as_index=False).size()
review_count = review_count.rename(columns={'size': 'review_count'})
reviews_merged = reviews_merged.drop_duplicates(['recipe_id'])
reviews_merged = pd.merge(reviews_merged, review_count, how='inner', on='recipe_id')
for recipe in recipes.id:
    if recipe not in reviews.recipe_id:
        print(recipe in reviews_merged.recipe_id)
        break

False


5.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [17]:
reviews_temp = reviews.groupby('recipe_id', as_index=False)['rating'].mean()
recipes_temp = recipes[['id', 'submitted']].copy()
reviews_temp = pd.merge(reviews_temp, recipes_temp, how='inner', left_on='recipe_id', right_on='id')
print(reviews_temp.groupby(recipes_temp.submitted.dt.year)['rating'].mean().idxmin())

2016


### Сохранение таблиц `pd.DataFrame`

6.1 Отсортируйте таблицу в порядке убывания величины столбца `name_word_count` и сохраните результаты выполнения заданий 3.1-3.3 в csv файл. 

In [18]:
recipes.sort_values('name_word_count', ascending=False).to_csv('data/recipes.csv')

6.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 5.1 и 5.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 5.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 5.2.

In [19]:
with pd.ExcelWriter('data/reviews.xlsx') as writer:
    recipes_merged.to_excel(writer, sheet_name='Рецепты с оценками')
    reviews_merged.to_excel(writer, sheet_name='Количество отзывов по рецептам')